In [2]:
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import dates as mdate
import json, datetime, pandas
pandas.options.display.float_format = "{:.2f}".format
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import ipywidgets as widgets 
from IPython.display import display, clear_output, Markdown

In [16]:
def lob_stats_analyser(df):
    outt = widgets.Output()
    stats_checkbox = [widgets.Checkbox(description=stats, value=True,indent=False, disabled=False) for stats in df.columns]
    stats_checkboxes = widgets.HBox(stats_checkbox)
    rolling = widgets.FloatSlider(
             value=50,
             min=1,
             max=200,
             step=1,
             disabled=False,
             description='Rolling:',
            )
    timerange = widgets.IntRangeSlider(
                    value=[0,0],
                    min=0,
                    max=0,
                    step=5,
                    description='Range:',
                    disabled=False,
                    continuous_update=False,
                    orientation='horizontal',
                    readout=True,
                    readout_format='d',
                    layout=dict(width='95%')
                    )
    timerange.max = df.index.max()
    timerange.min = df.index.min()
    timerange.value = [df.index.min(), df.index.max()]

    def plot_time_range(start, end, normalized):
        with outt:
            clear_output()
            timerange.value = [start, end]
            print("Start time:", datetime.datetime.fromtimestamp(start).strftime('%Y-%m-%d %H:%M:%S'),
                    "  End time:", datetime.datetime.fromtimestamp(end).strftime('%Y-%m-%d %H:%M:%S'))
            selected_stats = [checkbox.description for checkbox in stats_checkbox if checkbox.value]
            if normalized:
                data = df[(df.index > start) & 
                    (df.index < end)][selected_stats].rolling(int(rolling.value)).mean().dropna()
            else:
                data = df[(df.index > start) & 
                    (df.index < end)][selected_stats].rolling(int(rolling.value)).mean().dropna()
            plt.rcParams['figure.figsize'] = [15,7]
            plt.rcParams['timezone'] = 'America/Sao_Paulo'

            fig,ax = plt.subplots()
            x = mdate.epoch2num(data.index)
            fmt = mdate.DateFormatter('%H:%M:%S')

            ax.xaxis.set_major_formatter(fmt)
            plt.xticks( rotation=25 )
            plt.plot_date(x, data, linestyle='solid', marker='None')

            plt.legend(data.columns)
            plt.show()
    def on_btn_filter_range_clicked(b):
        plot_time_range(timerange.value[0], timerange.value[1], False)
    def on_btn_filter_range_norm_clicked(b):
        plot_time_range(timerange.value[0], timerange.value[1], True)
    btn_filter_range = widgets.Button(description='Apply Filter',button_style='danger', disabled=False)
    btn_filter_range.on_click(on_btn_filter_range_clicked)
    btn_filter_range_norm = widgets.Button(description='Apply Filter w/ Norm',button_style='warning', disabled=False)
    btn_filter_range_norm.on_click(on_btn_filter_range_norm_clicked)
    btns = widgets.HBox([btn_filter_range, btn_filter_range_norm])

    return widgets.VBox([stats_checkboxes, rolling, timerange,btns, outt])

In [17]:
df = pandas.read_csv('bpac11_bear_market_20210308.csv',sep=';')
df.set_index('timestamp', inplace=True)
df.head()

,spread,book_imbalance,best_bid_p,best_ask_p,pressure_ask,pressure_bid,wt_bid_p,wt_ask_p,wt_mid_p,wt_p,middle_p
timestamp,,,,,,,,,,,
1615212000,0.07,-0.14,94.53,94.60,0.57,0.43,94.44,94.73,94.54,94.61,94.56
1615212000,0.06,-0.04,94.53,94.59,0.52,0.48,94.44,94.72,94.54,94.59,94.56
1615212000,0.06,-0.09,94.53,94.59,0.54,0.46,94.44,94.72,94.54,94.59,94.56
1615212000,0.08,-0.14,94.51,94.59,0.57,0.43,94.43,94.71,94.53,94.59,94.55
1615212000,0.08,-0.12,94.51,94.59,0.56,0.44,94.43,94.71,94.52,94.59,94.55


In [18]:
lob_stats_analyser(df)

In [5]:
df.apply(lambda x: (x-x.min())/(x.max()-x.min()))

,spread,book_imbalance,best_bid_p,best_ask_p,pressure_ask,pressure_bid,wt_bid_p,wt_ask_p,wt_mid_p,wt_p,middle_p
timestamp,,,,,,,,,,,
1615212000,0.10,0.38,0.99,0.98,0.62,0.38,0.98,0.98,0.98,0.98,0.98
1615212000,0.10,0.44,0.99,0.98,0.56,0.44,0.98,0.98,0.98,0.98,0.98
1615212000,0.10,0.41,0.99,0.98,0.59,0.41,0.98,0.98,0.98,0.98,0.98
1615212000,0.10,0.38,0.99,0.98,0.62,0.38,0.98,0.98,0.98,0.98,0.98
1615212000,0.10,0.39,0.99,0.98,0.61,0.39,0.98,0.98,0.98,0.98,0.98
...,...,...,...,...,...,...,...,...,...,...,...
1615237580,0.10,0.26,0.94,0.87,0.74,0.26,0.87,0.84,0.87,0.85,0.88
1615237600,0.10,0.26,0.94,0.87,0.74,0.26,0.87,0.84,0.87,0.85,0.88
1615237607,0.10,0.27,0.94,0.87,0.73,0.27,0.87,0.84,0.87,0.85,0.88


In [7]:
dfn = df.apply(lambda x: (x-x.min())/(x.max()-x.min()))

In [9]:
dfn.head()

,spread,book_imbalance,best_bid_p,best_ask_p,pressure_ask,pressure_bid,wt_bid_p,wt_ask_p,wt_mid_p,wt_p,middle_p
timestamp,,,,,,,,,,,
1615212000,0.10,0.38,0.99,0.98,0.62,0.38,0.98,0.98,0.98,0.98,0.98
1615212000,0.10,0.44,0.99,0.98,0.56,0.44,0.98,0.98,0.98,0.98,0.98
1615212000,0.10,0.41,0.99,0.98,0.59,0.41,0.98,0.98,0.98,0.98,0.98
1615212000,0.10,0.38,0.99,0.98,0.62,0.38,0.98,0.98,0.98,0.98,0.98
1615212000,0.10,0.39,0.99,0.98,0.61,0.39,0.98,0.98,0.98,0.98,0.98


In [12]:
lob_stats_analyser(dfn)

TraitError: The 'button_style' trait of a Button instance must be any of ['primary', 'success', 'info', 'warning', 'danger', ''], but a value of 'Warn' <class 'str'> was specified.